# Threads Conversacionais

Muitas aplicações LLM têm uma interface similar a chatbot na qual o usuário e a aplicação LLM se envolvem em uma conversa de múltiplas rodadas. Para rastrear essas conversas, você pode usar o recurso Threads no LangSmith.

Isso é relevante para nossa aplicação RAG, que deve manter contexto de conversas anteriores com usuários.

### Configuração

In [ ]:
# Você pode defini-las diretamente
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # Se você não definir isso, os rastreamentos irão para o projeto Default

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Agrupar rastreamentos em threads

Um Thread é uma sequência de rastreamentos representando uma única conversa. Cada resposta é representada como seu próprio rastreamento, mas esses rastreamentos são ligados juntos por fazer parte do mesmo thread.

Para associar rastreamentos juntos, você precisa passar uma chave de metadados especial onde o valor é o identificador único para aquele thread.

O valor da chave é o identificador único para aquela conversa. O nome da chave deve ser um dos seguintes:

- session_id
- thread_id
- conversation_id.

O valor deve ser um UUID.

In [5]:
import uuid
thread_id = uuid.uuid4()

In [ ]:
from langsmith import traceable
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat

nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """Você é um assistente para tarefas de perguntas e respostas.
    Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.
    Se você não souber a resposta, apenas diga que não sabe.
    Use no máximo três frases e mantenha a resposta concisa.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_gemini(messages)

@traceable(
    run_type="llm",
    metadata={
        "ls_provider": "google",
        "ls_model_name": "gemini-1.5-flash"
    }
)
def call_gemini(
    messages: List[dict], model: str = "gemini-1.5-flash", temperature: float = 0.0
) -> str:
    return call_gemini_chat(model, messages, temperature)

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

### Agora vamos executar nossa aplicação duas vezes com este thread_id

In [ ]:
question = "Como adiciono metadados a um Rastreamento?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

In [ ]:
question = "Como posso adicionar tags a um Rastreamento?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

### Vamos dar uma olhada no LangSmith!